# New Notebook file

In [ ]:
import sqlite3

import pandas as pd

# 1. Connect
conn = sqlite3.connect("data/olist.db")

# 2. Query: Get dates and location
# We filter for 'delivered' orders only to avoid NULL delivery dates
query = """
SELECT 
    c.customer_state,
    o.order_purchase_timestamp,
    o.order_delivered_customer_date
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.order_status = 'delivered'
"""

print("Fetching data...")
df = pd.read_sql(query, conn)
conn.close()

# 3. Data Processing (Pandas is great for Date Math)
# Convert string dates to datetime objects
df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"])
df["order_delivered_customer_date"] = pd.to_datetime(df["order_delivered_customer_date"])

# Calculate duration in days
df["delivery_days"] = (df["order_delivered_customer_date"] - df["order_purchase_timestamp"]).dt.days

# 4. Aggregation
# Group by State and get the mean delivery time
avg_delivery = df.groupby("customer_state")["delivery_days"].mean().sort_values()

print("\nAverage Delivery Time by State (Days):")
print(avg_delivery)

# Optional: Show the fastest and slowest
print(f"\nFastest State: {avg_delivery.index[0]} ({avg_delivery.iloc[0]:.2f} days)")
print(f"Slowest State: {avg_delivery.index[-1]} ({avg_delivery.iloc[-1]:.2f} days)")